# CUDA SmallPT Demo

CUDA C++ modification of Kevin Baeson's [smallpt](http://www.kevinbeason.com/smallpt/).

Adapted from https://github.com/matt77hias/cu-smallpt.git

## Main helpers

In [1]:
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <algorithm>
#include <cmath>
#include <cstdint>
#include <limits>
#include <cstdio>
#include <cstdlib>

namespace smallpt {

  constexpr double g_pi = 3.14159265358979323846;

  __host__ __device__ inline double Clamp(double v, 
                      double low = 0.0, 
                      double high = 1.0) noexcept
  {
    return fmin(fmax(v, low), high);
  }

  inline std::uint8_t ToByte(double color, double gamma = 2.2) noexcept
  {
    const double gcolor = std::pow(color, 1.0 / gamma);
    return static_cast< std::uint8_t >(Clamp(255.0 * gcolor, 0.0, 255.0));
  }

  inline void HandleError(cudaError_t err, const char* file, int line) {
    if (cudaSuccess != err) {
      std::printf("%s in %s at line %d\n", 
            cudaGetErrorString(err), file, line);
      std::exit(EXIT_FAILURE);
    }
  }
}

#define HANDLE_ERROR(err) (HandleError(err, __FILE__, __LINE__))

#define HANDLE_NULL(a) {if (a == NULL) { \
  std::printf("Host memory failed in %s at line %d\n", __FILE__, __LINE__); \
    std::exit(EXIT_FAILURE);}}

## Vector helpers

In [2]:
//#include "math.hpp"

namespace smallpt {

  struct Vector3 {

  public:

    __host__ __device__ explicit Vector3(double xyz = 0.0) noexcept
      : Vector3(xyz, xyz, xyz) {}
    __host__ __device__ Vector3(double x, double y, double z) noexcept
      : m_x(x), m_y(y), m_z(z) {}
    Vector3(const Vector3& v) noexcept = default;
    Vector3(Vector3&& v) noexcept = default;
    ~Vector3() = default;

    Vector3& operator=(const Vector3& v) = default;
    Vector3& operator=(Vector3&& v) = default;

    __device__ bool HasNaNs() const {
      return std::isnan(m_x) || std::isnan(m_y) || std::isnan(m_z);
    }

    __device__ const Vector3 operator-() const {
      return { -m_x, -m_y, -m_z };
    }

    __device__ const Vector3 operator+(const Vector3& v) const {
      return { m_x + v.m_x, m_y + v.m_y, m_z + v.m_z };
    }
    __device__ const Vector3 operator-(const Vector3& v) const {
      return { m_x - v.m_x, m_y - v.m_y, m_z - v.m_z };
    }
    __device__ const Vector3 operator*(const Vector3& v) const {
      return { m_x * v.m_x, m_y * v.m_y, m_z * v.m_z };
    }
    __device__ const Vector3 operator/(const Vector3& v) const {
      return { m_x / v.m_x, m_y / v.m_y, m_z / v.m_z };
    }
    __device__ const Vector3 operator+(double a) const {
      return { m_x + a, m_y + a, m_z + a };
    }
    __device__ const Vector3 operator-(double a) const {
      return { m_x - a, m_y - a, m_z - a };
    }
    __device__ const Vector3 operator*(double a) const {
      return { m_x * a, m_y * a, m_z * a };
    }
    __device__ const Vector3 operator/(double a) const {
      const double inv_a = 1.0 / a;
      return { m_x * inv_a, m_y * inv_a, m_z * inv_a };
    }

    __device__ Vector3& operator+=(const Vector3& v) {
      m_x += v.m_x;
      m_y += v.m_y;
      m_z += v.m_z;
      return *this;
    }
    __device__ Vector3& operator-=(const Vector3& v) {
      m_x -= v.m_x;
      m_y -= v.m_y;
      m_z -= v.m_z;
      return *this;
    }
    __device__ Vector3& operator*=(const Vector3& v) {
      m_x *= v.m_x;
      m_y *= v.m_y;
      m_z *= v.m_z;
      return *this;
    }
    __device__ Vector3& operator/=(const Vector3& v) {
      m_x /= v.m_x;
      m_y /= v.m_y;
      m_z /= v.m_z;
      return *this;
    }
    __device__ Vector3& operator+=(double a) {
      m_x += a;
      m_y += a;
      m_z += a;
      return *this;
    }
    __device__ Vector3& operator-=(double a) {
      m_x -= a;
      m_y -= a;
      m_z -= a;
      return *this;
    }
    __device__ Vector3& operator*=(double a) {
      m_x *= a;
      m_y *= a;
      m_z *= a;
      return *this;
    }
    __device__ Vector3& operator/=(double a) {
      const double inv_a = 1.0 / a;
      m_x *= inv_a;
      m_y *= inv_a;
      m_z *= inv_a;
      return *this;
    }

    __device__ double Dot(const Vector3& v) const {
      return m_x * v.m_x + m_y * v.m_y + m_z * v.m_z;
    }
    __device__ const Vector3 Cross(const Vector3& v) const {
      return {
        m_y * v.m_z - m_z * v.m_y,
        m_z * v.m_x - m_x * v.m_z,
        m_x * v.m_y - m_y * v.m_x
      };
    }

    __device__ bool operator==(const Vector3& rhs) const {
      return m_x == rhs.m_x && m_y == rhs.m_y && m_z == rhs.m_z;
    }
    __device__ bool operator!=(const Vector3& rhs) const {
      return !(*this == rhs);
    }

    __device__ double& operator[](std::size_t i) {
      return (&m_x)[i];
    }
    __device__ double operator[](std::size_t i) const {
      return (&m_x)[i];
    }

    __device__ std::size_t MinDimension() const {
      return (m_x < m_y && m_x < m_z) ? 0u : ((m_y < m_z) ? 1u : 2u);
    }
    __device__ std::size_t MaxDimension() const {
      return (m_x > m_y && m_x > m_z) ? 0u : ((m_y > m_z) ? 1u : 2u);
    }
    __device__ double Min() const {
      return fmin(m_x, fmin(m_y, m_z));
    }
    __device__ double Max() const {
      return fmax(m_x, fmax(m_y, m_z));
    }

    __device__ double Norm2_squared() const {
      return m_x * m_x + m_y * m_y + m_z * m_z;
    }

    __device__ double Norm2() const {
      return std::sqrt(Norm2_squared());
    }

    __device__ void Normalize() {
      const double a = 1.0 / Norm2();
      m_x *= a;
      m_y *= a;
      m_z *= a;
    }

    double m_x, m_y, m_z;
  };

  __device__ inline const Vector3 operator+(double a, const Vector3& v) {
    return { a + v.m_x, a + v.m_y, a + v.m_z };
  }

  __device__ inline const Vector3 operator-(double a, const Vector3& v) {
    return { a - v.m_x, a - v.m_y, a - v.m_z };
  }

  __device__ inline const Vector3 operator*(double a, const Vector3& v) {
    return { a * v.m_x, a * v.m_y, a * v.m_z };
  }

  __device__ inline const Vector3 operator/(double a, const Vector3& v) {
    return { a / v.m_x, a / v.m_y, a / v.m_z };
  }

  __device__ inline const Vector3 Sqrt(const Vector3& v) {
    return {
      std::sqrt(v.m_x),
      std::sqrt(v.m_y),
      std::sqrt(v.m_z)
    };
  }

  __device__ inline const Vector3 Pow(const Vector3& v, double a) {
    return {
      std::pow(v.m_x, a),
      std::pow(v.m_y, a),
      std::pow(v.m_z, a)
    };
  }

  __device__ inline const Vector3 Abs(const Vector3& v) {
    return {
      std::abs(v.m_x),
      std::abs(v.m_y),
      std::abs(v.m_z)
    };
  }

  __device__ inline const Vector3 Min(const Vector3& v1, const Vector3& v2) {
    return {
      fmin(v1.m_x, v2.m_x),
      fmin(v1.m_y, v2.m_y),
      fmin(v1.m_z, v2.m_z)
    };
  }

  __device__ inline const Vector3 Max(const Vector3& v1, const Vector3& v2) {
    return {
      fmax(v1.m_x, v2.m_x),
      fmax(v1.m_y, v2.m_y),
      fmax(v1.m_z, v2.m_z)
    };
  }

  __device__ inline const Vector3 Round(const Vector3& v) {
    return {
      std::round(v.m_x),
      std::round(v.m_y),
      std::round(v.m_z)
    };
  }

  __device__ inline const Vector3 Floor(const Vector3& v) {
    return {
      std::floor(v.m_x),
      std::floor(v.m_y),
      std::floor(v.m_z)
    };
  }

  __device__ inline const Vector3 Ceil(const Vector3& v) {
    return {
      std::ceil(v.m_x),
      std::ceil(v.m_y),
      std::ceil(v.m_z)
    };
  }

  __device__ inline const Vector3 Trunc(const Vector3& v) {
    return {
      std::trunc(v.m_x),
      std::trunc(v.m_y),
      std::trunc(v.m_z)
    };
  }

  __device__ inline const Vector3 Clamp(const Vector3& v, 
                      double low = 0.0, 
                      double high = 1.0) {
    return {
      Clamp(v.m_x, low, high),
      Clamp(v.m_y, low, high),
      Clamp(v.m_z, low, high) }
    ;
  }

  __device__ inline const Vector3 Lerp(double a, 
                     const Vector3& v1, 
                     const Vector3& v2) {
    return v1 + a * (v2 - v1);
  }

  template< std::size_t X, std::size_t Y, std::size_t Z >
  __device__ inline const Vector3 Permute(const Vector3& v) {
    return { v[X], v[Y], v[Z] };
  }

  __device__ inline const Vector3 Normalize(const Vector3& v) {
    const double a = 1.0 / v.Norm2();
    return a * v;
  }
}

## Image IO helper

In [3]:
//#include "vector.hpp"
#include <cstdio>

#ifdef __unix
#define fopen_s(pFile,filename,mode) ((*(pFile))=fopen((filename),(mode)))==NULL
#endif

namespace smallpt {

  inline void WritePPM(std::uint32_t w, 
             std::uint32_t h, 
             const Vector3* Ls, 
             const char* fname = "cu-image.ppm") noexcept {
    
    FILE* fp;
    
    fopen_s(&fp, fname, "w");
    
    std::fprintf(fp, "P3\n%u %u\n%u\n", w, h, 255u);
    for (std::size_t i = 0; i < w * h; ++i) {
      std::fprintf(fp, "%u %u %u ", 
             ToByte(Ls[i].m_x), 
             ToByte(Ls[i].m_y), 
             ToByte(Ls[i].m_z));
    }
    
    std::fclose(fp);
  }
}

### SV PNG

In [4]:
/*
Copyright (C) 2017 Milo Yip. All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

* Redistributions of source code must retain the above copyright notice, this
  list of conditions and the following disclaimer.

* Redistributions in binary form must reproduce the above copyright notice,
  this list of conditions and the following disclaimer in the documentation
  and/or other materials provided with the distribution.

* Neither the name of pngout nor the names of its
  contributors may be used to endorse or promote products derived from
  this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
*/

/*! \file
    \brief      svpng() is a minimalistic C function for saving RGB/RGBA image into uncompressed PNG.
    \author     Milo Yip
    \version    0.1.1
    \copyright  MIT license
    \sa         http://github.com/miloyip/svpng
*/

///#ifndef SVPNG_INC_
///#define SVPNG_INC_

/*! \def SVPNG_LINKAGE
    \brief User customizable linkage for svpng() function.
    By default this macro is empty.
    User may define this macro as static for static linkage, 
    and/or inline in C99/C++, etc.
*/
///#ifndef SVPNG_LINKAGE
#define SVPNG_LINKAGE
///#endif

/*! \def SVPNG_OUTPUT
    \brief User customizable output stream.
    By default, it uses C file descriptor and fputc() to output bytes.
    In C++, for example, user may use std::ostream or std::vector instead.
*/
///#ifndef SVPNG_OUTPUT
#include <stdio.h>
#define SVPNG_OUTPUT FILE* fp
///#endif

/*! \def SVPNG_PUT
    \brief Write a byte
*/
///#ifndef SVPNG_PUT
#define SVPNG_PUT(u) fputc(u, fp)
///#endif


/*!
    \brief Save a RGB/RGBA image in PNG format.
    \param SVPNG_OUTPUT Output stream (by default using file descriptor).
    \param w Width of the image. (<16383)
    \param h Height of the image.
    \param img Image pixel data in 24-bit RGB or 32-bit RGBA format.
    \param alpha Whether the image contains alpha channel.
*/
SVPNG_LINKAGE void svpng(SVPNG_OUTPUT, unsigned w, unsigned h, const unsigned char* img, int alpha) {
    static const unsigned t[] = { 0, 0x1db71064, 0x3b6e20c8, 0x26d930ac, 0x76dc4190, 0x6b6b51f4, 0x4db26158, 0x5005713c, 
    /* CRC32 Table */    0xedb88320, 0xf00f9344, 0xd6d6a3e8, 0xcb61b38c, 0x9b64c2b0, 0x86d3d2d4, 0xa00ae278, 0xbdbdf21c };
    unsigned a = 1, b = 0, c, p = w * (alpha ? 4 : 3) + 1, x, y, i;   /* ADLER-a, ADLER-b, CRC, pitch */
#define SVPNG_U8A(ua, l) for (i = 0; i < l; i++) SVPNG_PUT((ua)[i]);
#define SVPNG_U32(u) do { SVPNG_PUT((u) >> 24); SVPNG_PUT(((u) >> 16) & 255); SVPNG_PUT(((u) >> 8) & 255); SVPNG_PUT((u) & 255); } while(0)
#define SVPNG_U8C(u) do { SVPNG_PUT(u); c ^= (u); c = (c >> 4) ^ t[c & 15]; c = (c >> 4) ^ t[c & 15]; } while(0)
#define SVPNG_U8AC(ua, l) for (i = 0; i < l; i++) SVPNG_U8C((ua)[i])
#define SVPNG_U16LC(u) do { SVPNG_U8C((u) & 255); SVPNG_U8C(((u) >> 8) & 255); } while(0)
#define SVPNG_U32C(u) do { SVPNG_U8C((u) >> 24); SVPNG_U8C(((u) >> 16) & 255); SVPNG_U8C(((u) >> 8) & 255); SVPNG_U8C((u) & 255); } while(0)
#define SVPNG_U8ADLER(u) do { SVPNG_U8C(u); a = (a + (u)) % 65521; b = (b + a) % 65521; } while(0)
#define SVPNG_BEGIN(s, l) do { SVPNG_U32(l); c = ~0U; SVPNG_U8AC(s, 4); } while(0)
#define SVPNG_END() SVPNG_U32(~c)
    SVPNG_U8A("\x89PNG\r\n\32\n", 8);           /* Magic */
    SVPNG_BEGIN("IHDR", 13);                    /* IHDR chunk { */
    SVPNG_U32C(w); SVPNG_U32C(h);               /*   Width & Height (8 bytes) */
    SVPNG_U8C(8); SVPNG_U8C(alpha ? 6 : 2);     /*   Depth=8, Color=True color with/without alpha (2 bytes) */
    SVPNG_U8AC("\0\0\0", 3);                    /*   Compression=Deflate, Filter=No, Interlace=No (3 bytes) */
    SVPNG_END();                                /* } */
    SVPNG_BEGIN("IDAT", 2 + h * (5 + p) + 4);   /* IDAT chunk { */
    SVPNG_U8AC("\x78\1", 2);                    /*   Deflate block begin (2 bytes) */
    for (y = 0; y < h; y++) {                   /*   Each horizontal line makes a block for simplicity */
        SVPNG_U8C(y == h - 1);                  /*   1 for the last block, 0 for others (1 byte) */
        SVPNG_U16LC(p); SVPNG_U16LC(~p);        /*   Size of block in little endian and its 1's complement (4 bytes) */
        SVPNG_U8ADLER(0);                       /*   No filter prefix (1 byte) */
        for (x = 0; x < p - 1; x++, img++)
            SVPNG_U8ADLER(*img);                /*   Image pixel data */
    }
    SVPNG_U32C((b << 16) | a);                  /*   Deflate block end with adler (4 bytes) */
    SVPNG_END();                                /* } */
    SVPNG_BEGIN("IEND", 0); SVPNG_END();        /* IEND chunk {} */
}

///#endif /* SVPNG_INC_ */

### Image helper

In [5]:
int toInt(float x)
{
    return (int) (pow(smallpt::Clamp(x, 0.0f, 1.0f), 1.0f / 2.2f) * 255 + 0.5f);
}

void save(const char* fileName, int width, int height, smallpt::Vector3* data)
{
    FILE *fp = fopen(fileName, "wb");

    // Convert from float3 array to uchar array
    unsigned char* output = new unsigned char[width * height * 3];

    for(int i = 0; i < width * height; i++)
    {
        //printf_s("%f %f %f \n", data[i].x, data[i].y, data[i].z);
        output[i * 3 + 0] = toInt(data[i].m_x);
        output[i * 3 + 1] = toInt(data[i].m_y);
        output[i * 3 + 2] = toInt(data[i].m_z);
    }

    svpng(fp, width, height, output, 0);
    fclose(fp);
    delete[] output;
}

//#include "xeus/xjson.hpp"
/*
inline std::string base64encode(const std::string& input)
{
  std::string output;
  int val = 0;
  int valb = -6;
  for (char sc : input)
  {
    unsigned char c = static_cast<unsigned char>(sc);
    val = (val << 8) + c;
    valb += 8;
    while (valb >= 0) {
        output.push_back("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/"[(val >> valb) & 0x3F]);
        valb -= 6;
    }
  }
  if (valb > -6) {
      output.push_back("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/"[((val << 8) >> (valb + 8)) & 0x3F]);
  }
  while (output.size() % 4) {
      output.push_back('=');
  }
  return output;
}
*/
/*
// display image in the notebook
void display_image(std::vector<unsigned char> &image, bool clear_ouput){
    // memory objects for output in the web browser
    std::stringstream buffer;
    xeus::xjson mine;

    if(clear_ouput)
        xeus::get_interpreter().clear_output(true);

    buffer.str("");
    for(auto c : image){
        buffer << c;
    }

    mine["image/png"] = xtl::base64encode(buffer.str());
    xeus::get_interpreter().display_data(
        std::move(mine),
        xeus::xjson::object(),
        xeus::xjson::object());
}
*/

# CUDA SmallPT main

In [ ]:
//#include "imageio.hpp"
//#include "sampling.cuh"
//#include "specular.cuh"
//#include "sphere.hpp"

//#include "cuda_tools.hpp"
#include "curand_kernel.h"

#define REFRACTIVE_INDEX_OUT 1.0
#define REFRACTIVE_INDEX_IN  1.5
#define EPSILON_SPHERE 1e-4

namespace smallpt {

  struct __align__(16)Ray {
    __device__ explicit Ray(Vector3 o, 
                Vector3 d, 
                double tmin = 0.0, 
                double tmax = std::numeric_limits< double >::infinity(), 
                std::uint32_t depth = 0u) noexcept
      : m_o(std::move(o)),
      m_d(std::move(d)),
      m_tmin(tmin),
      m_tmax(tmax),
      m_depth(depth) {};
    Ray(const Ray& ray) noexcept = default;
    Ray(Ray&& ray) noexcept = default;
    ~Ray() = default;

    Ray& operator=(const Ray& ray) = default;
    Ray& operator=(Ray&& ray) = default;

    __device__ const Vector3 operator()(double t) const {
      return m_o + m_d * t;
    }

    Vector3 m_o, m_d;
    mutable double m_tmin, m_tmax;
    std::uint32_t m_depth;
  };

  enum struct Reflection_t {
    Diffuse,
    Specular,
    Refractive
  };

  struct __align__(16) Sphere {
    __host__ __device__ explicit Sphere(double r,
                      Vector3 p,
                      Vector3 e,
                      Vector3 f,
                      Reflection_t reflection_t) noexcept
      : m_r(r),
      m_p(std::move(p)),
      m_e(std::move(e)),
      m_f(std::move(f)),
      m_reflection_t(reflection_t) {}
    Sphere(const Sphere& sphere) noexcept = default;
    Sphere(Sphere&& sphere) noexcept = default;
    ~Sphere() = default;

    Sphere& operator=(const Sphere& sphere) = default;
    Sphere& operator=(Sphere&& sphere) = default;
    
    __device__ bool Intersect(const Ray& ray) const {
      const Vector3 op = m_p - ray.m_o;
      const double dop = ray.m_d.Dot(op);
      const double D = dop * dop - op.Dot(op) + m_r * m_r;

      if (0.0 > D) {
        return false;
      }

      const double sqrtD = sqrt(D);

      const double tmin = dop - sqrtD;
      if (ray.m_tmin < tmin && tmin < ray.m_tmax) {
        ray.m_tmax = tmin;
        return true;
      }

      const double tmax = dop + sqrtD;
      if (ray.m_tmin < tmax && tmax < ray.m_tmax) {
        ray.m_tmax = tmax;
        return true;
      }

      return false;
    }

    double m_r;
    Vector3 m_p; // position
    Vector3 m_e; // emission
    Vector3 m_f; // reflection
    Reflection_t m_reflection_t;
  };
    
  const Sphere g_spheres[] = {
    Sphere(1e5,  Vector3(1e5 + 1, 40.8, 81.6),   Vector3(),   Vector3(0.75,0.25,0.25), Reflection_t::Diffuse),    //Left
    Sphere(1e5,  Vector3(-1e5 + 99, 40.8, 81.6), Vector3(),   Vector3(0.25,0.25,0.75), Reflection_t::Diffuse),    //Right
    Sphere(1e5,  Vector3(50, 40.8, 1e5),         Vector3(),   Vector3(0.75),           Reflection_t::Diffuse),    //Back
    Sphere(1e5,  Vector3(50, 40.8, -1e5 + 170),  Vector3(),   Vector3(),               Reflection_t::Diffuse),    //Front
    Sphere(1e5,  Vector3(50, 1e5, 81.6),         Vector3(),   Vector3(0.75),           Reflection_t::Diffuse),    //Bottom
    Sphere(1e5,  Vector3(50, -1e5 + 81.6, 81.6), Vector3(),   Vector3(0.75),           Reflection_t::Diffuse),    //Top
    Sphere(16.5, Vector3(27, 16.5, 47),          Vector3(),   Vector3(0.999),          Reflection_t::Specular),   //Mirror
    Sphere(16.5, Vector3(73, 16.5, 78),          Vector3(),   Vector3(0.999),          Reflection_t::Refractive), //Glass
    Sphere(600,  Vector3(50, 681.6 - .27, 81.6), Vector3(12), Vector3(),               Reflection_t::Diffuse)     //Light
  };

  __device__ inline Vector3 UniformSampleOnHemisphere(double u1,  double u2) {
    // u1 := cos_theta
    const double sin_theta = std::sqrt(fmax(0.0, 1.0 - u1 * u1));
    const double phi = 2.0 * g_pi * u2;
    return {
      std::cos(phi) * sin_theta,
      std::sin(phi) * sin_theta,
      u1
    };
  }

  __device__ inline Vector3 CosineWeightedSampleOnHemisphere(double u1, double u2) {
    const double cos_theta = sqrt(1.0 - u1);
    const double sin_theta = sqrt(u1);
    const double phi = 2.0 * g_pi * u2;
    return {
      std::cos(phi) * sin_theta,
      std::sin(phi) * sin_theta,
      cos_theta
    };
  }
    
  __device__ inline double Reflectance0(double n1, double n2) {
    const double sqrt_R0 = (n1 - n2) / (n1 + n2);
    return sqrt_R0 * sqrt_R0;
  }

  __device__ inline double SchlickReflectance(double n1, 
                        double n2, 
                        double c) {
    const double R0 = Reflectance0(n1, n2);
    return R0 + (1.0 - R0) * c * c * c * c * c;
  }

  __device__ inline const Vector3 IdealSpecularReflect(const Vector3& d, const Vector3& n) {
    return d - 2.0 * n.Dot(d) * n;
  }

  __device__ inline const Vector3 IdealSpecularTransmit(const Vector3& d, 
                              const Vector3& n, 
                              double n_out, 
                              double n_in, 
                              double& pr, 
                              curandState* state) {
    
    const Vector3 d_Re = IdealSpecularReflect(d, n);

    const bool out_to_in = (0.0 > n.Dot(d));
    const Vector3 nl = out_to_in ? n : -n;
    const double nn = out_to_in ? n_out / n_in : n_in / n_out;
    const double cos_theta = d.Dot(nl);
    const double cos2_phi = 1.0 - nn * nn * (1.0 - cos_theta * cos_theta);

    // Total Internal Reflection
    if (0.0 > cos2_phi) {
      pr = 1.0;
      return d_Re;
    }

    const Vector3 d_Tr = Normalize(nn * d - nl * (nn * cos_theta + sqrt(cos2_phi)));
    const double c = 1.0 - (out_to_in ? -cos_theta : d_Tr.Dot(n));

    const double Re = SchlickReflectance(n_out, n_in, c);
    const double p_Re = 0.25 + 0.5 * Re;
    if (curand_uniform_double(state) < p_Re) {
      pr = (Re / p_Re);
      return d_Re;
    }
    else {
      const double Tr = 1.0 - Re;
      const double p_Tr = 1.0 - p_Re;
      pr = (Tr / p_Tr);
      return d_Tr;
    }
  }

  __device__ inline bool Intersect(const Sphere* dev_spheres, 
                   std::size_t nb_spheres, 
                   const Ray& ray, 
                   size_t& id)
  {
    bool hit = false;
    for (std::size_t i = 0u; i < nb_spheres; ++i) {
      if (dev_spheres[i].Intersect(ray)) {
        hit = true;
        id  = i;
      }
    }

    return hit;
  }

  __device__ static Vector3 Radiance(const Sphere* dev_spheres, 
                     std::size_t nb_spheres,
                     const Ray& ray, 
                     curandState* state)
  {  
    Ray r = ray;
    Vector3 L;
    Vector3 F(1.0);

    while (true) {
      std::size_t id;
      if (!Intersect(dev_spheres, nb_spheres, r, id)) {
        return L;
      }

      const Sphere& shape = dev_spheres[id];
      const Vector3 p = r(r.m_tmax);
      const Vector3 n = Normalize(p - shape.m_p);

      L += F * shape.m_e;
      F *= shape.m_f;

      // Russian roulette
      if (4 < r.m_depth) {
        const double continue_probability = shape.m_f.Max();
        if (curand_uniform_double(state) >= continue_probability) {
          return L;
        }
        F /= continue_probability;
      }

      // Next path segment
      switch (shape.m_reflection_t) {
      
      case Reflection_t::Specular: {
        const Vector3 d = IdealSpecularReflect(r.m_d, n);
        r = Ray(p, d, EPSILON_SPHERE, INFINITY, r.m_depth + 1u);
        break;
      }
      
      case Reflection_t::Refractive: {
        double pr;
        const Vector3 d = IdealSpecularTransmit(r.m_d, n, REFRACTIVE_INDEX_OUT, REFRACTIVE_INDEX_IN, pr, state);
        F *= pr;
        r = Ray(p, d, EPSILON_SPHERE, INFINITY, r.m_depth + 1u);
        break;
      }
      
      default: {
        const Vector3 w = (0.0 > n.Dot(r.m_d)) ? n : -n;
        const Vector3 u = Normalize((abs(w.m_x) > 0.1 ? Vector3(0.0, 1.0, 0.0) : Vector3(1.0, 0.0, 0.0)).Cross(w));
        const Vector3 v = w.Cross(u);

        const Vector3 sample_d = CosineWeightedSampleOnHemisphere(curand_uniform_double(state), curand_uniform_double(state));
        const Vector3 d = Normalize(sample_d.m_x * u + sample_d.m_y * v + sample_d.m_z * w);
        r = Ray(p, d, EPSILON_SPHERE, INFINITY, r.m_depth + 1u);
      }
      }
    }
  }

  __global__ static void kernel(const Sphere* dev_spheres, 
                  std::size_t nb_spheres,
                  std::uint32_t w, 
                  std::uint32_t h, 
                  Vector3* Ls, 
                  std::uint32_t nb_samples)
  {
    const std::uint32_t x = threadIdx.x + blockIdx.x * blockDim.x;
    const std::uint32_t y = threadIdx.y + blockIdx.y * blockDim.y;
    const std::uint32_t offset = x + y * blockDim.x * gridDim.x;

    if (x >= w || y >= h) {
      return;
    }

    // RNG
    curandState state;
    curand_init(offset, 0u, 0u, &state);

    const Vector3 eye = { 50.0, 52.0, 295.6 };
    const Vector3 gaze = Normalize(Vector3(0.0, -0.042612, -1.0));
    const double fov = 0.5135;
    const Vector3 cx = { w * fov / h, 0.0, 0.0 };
    const Vector3 cy = Normalize(cx.Cross(gaze)) * fov;

    for (std::size_t sy = 0u, i = (h - 1u - y) * w + x; sy < 2u; ++sy) { // 2 subpixel row

      for (std::size_t sx = 0u; sx < 2u; ++sx) { // 2 subpixel column

        Vector3 L;

        for (std::size_t s = 0u; s < nb_samples; ++s) { // samples per subpixel
          const double u1 = 2.0 * curand_uniform_double(&state);
          const double u2 = 2.0 * curand_uniform_double(&state);
          const double dx = (u1 < 1.0) ? sqrt(u1) - 1.0 : 1.0 - sqrt(2.0 - u1);
          const double dy = (u2 < 1.0) ? sqrt(u2) - 1.0 : 1.0 - sqrt(2.0 - u2);
          const Vector3 d = cx * (((sx + 0.5 + dx) * 0.5 + x) / w - 0.5) +
                          cy * (((sy + 0.5 + dy) * 0.5 + y) / h - 0.5) + gaze;
          
          L += Radiance(dev_spheres, nb_spheres, 
            Ray(eye + d * 130, Normalize(d), EPSILON_SPHERE), &state) 
            * (1.0 / nb_samples);
        }
        
        Ls[i] += 0.25 * Clamp(L);
      }
    }
  }

  static void Render(std::uint32_t nb_samples) noexcept
  {
    const std::uint32_t w = 256u; //1024u;
    const std::uint32_t h = 192u; //768u;
    const std::uint32_t nb_pixels = w * h;

    // Set up device memory
    Sphere* dev_spheres;
    HANDLE_ERROR(cudaMalloc((void**)&dev_spheres, sizeof(g_spheres)));
    HANDLE_ERROR(cudaMemcpy(dev_spheres, g_spheres, sizeof(g_spheres), cudaMemcpyHostToDevice));
    Vector3* dev_Ls;
    HANDLE_ERROR(cudaMalloc((void**)&dev_Ls, nb_pixels * sizeof(Vector3)));
    HANDLE_ERROR(cudaMemset(dev_Ls, 0, nb_pixels * sizeof(Vector3)));

    // Kernel execution
    const dim3 nblocks(w / 16u, h / 16u);
    const dim3 nthreads(16u, 16u);
    kernel<<<nblocks, nthreads>>>(dev_spheres, sizeof(g_spheres)/sizeof(g_spheres[0]),  w, h, dev_Ls, nb_samples);

    // Set up host memory
    Vector3* Ls = (Vector3*)malloc(nb_pixels * sizeof(Vector3));
    // Transfer device -> host
    HANDLE_ERROR(cudaMemcpy(Ls, dev_Ls, nb_pixels * sizeof(Vector3), cudaMemcpyDeviceToHost));

    // Clean up device memory
    HANDLE_ERROR(cudaFree(dev_Ls));
    HANDLE_ERROR(cudaFree(dev_spheres));

    //WritePPM(w, h, Ls);
    save("cu-smallpt.png", w, h, Ls);

    // Clean up host memory
    free(Ls);
  }
}

void devicePropertyPrint()
{
  int dev = 0;
  cudaDeviceProp devProp;
  if(cudaGetDeviceProperties(&devProp, dev) == cudaSuccess)
  {
    printf("Device %i, named: %s\n", dev, devProp.name);
    printf("Device compute capability: %i - %i\n", devProp.minor, devProp.major);
    printf("Device maxThreadDim: [%i, %i, %i]\n", devProp.maxThreadsDim[0], devProp.maxThreadsDim[1], devProp.maxThreadsDim[2]);
    printf("Device maxGridSize: [%i, %i, %i]\n", devProp.maxGridSize[0], devProp.maxGridSize[1], devProp.maxGridSize[2]);
    printf("Multi Processor Count: %i\n", devProp.multiProcessorCount);
    printf("Size of SharedMem Per-Block: %f KB\n", devProp.sharedMemPerBlock / 1024.0);
    printf("Max Threads Per-Block: %i\n", devProp.maxThreadsPerBlock);
    printf("Max Threads Per-MultiProcessor: %i\n", devProp.maxThreadsPerMultiProcessor);
  }
}

devicePropertyPrint();
const std::uint32_t nb_samples = 100;
smallpt::Render(nb_samples);

<img src="/files/notebooks/cu-smallpt.png" />

In [ ]:
%%python
from IPython.display import Image

In [ ]:
%%python
Image(filename='cu-smallpt.png')

In [ ]:
#include <fstream>
#include "nlohmann/json.hpp"
#include "xtl/xbase64.hpp"
#include "xeus/xinterpreter.hpp"

In [ ]:
void show_image() {
    std::ifstream fin("cu-smallpt.png", std::ios::binary);   
    std::stringstream m_buffer;
    m_buffer << fin.rdbuf();
    auto bundle = nlohmann::json::object();
    bundle["image/png"] = xtl::base64encode(m_buffer.str());
    ////
    //bundle["text/html"] = "<p>test</p>";
    //       std::string("<img src='data:image/png;base64,")
    //       + xtl::base64encode(m_buffer.str()) +
    //        "' width=256 height=192 />";
    ////
    //printf("%s\n", bundle.dump().c_str());
    xeus::get_interpreter().display_data(std::move(bundle), nlohmann::json::object(), nlohmann::json::object());
}

In [ ]:
show_image();